# Preambule
#### Goal
The goal of this session is to get familiar with the Bloomberg Python API.<br> 
This will be done by building a class containing a function which mimicks the behavior of the BDH Excel function.

#### What the function will do
Our BDH-like function should be able to : <br>
1 - Retrieve historical data <br>
2 - For as many tickers as possible <br>
3 - For as many fields as possible <br>
4 - Include the various options <br>
5 - And allow for the possibility to add overrides <br>

#### References 
https://data.bloomberglp.com/professional/sites/10/2017/03/BLPAPI-Core-Developer-Guide.pdf

# I. Dependencies
These are the libraries we will be using in this notebook. blpapi is the library used for Bloomberg data.

In [1]:
import blpapi
import pandas as pd
import numpy as np
import datetime as dt

# II. Set up the Bloomberg names

We here create variables using the Name class within blpapi. <br> 
This will allow to write cleaner and more concise code when refering to strings with the api.<br>
Below are only the names required for our present work. Many more exist and you can refer to the different examples within the SDK for ones of interest to your task.

In [2]:
DATE = blpapi.Name("date")
ERROR_INFO = blpapi.Name("errorInfo")
EVENT_TIME = blpapi.Name("EVENT_TIME")
FIELD_DATA = blpapi.Name("fieldData")
FIELD_EXCEPTIONS = blpapi.Name("fieldExceptions")
FIELD_ID = blpapi.Name("fieldId")
SECURITY = blpapi.Name("security")
SECURITY_DATA = blpapi.Name("securityData")

# III. The BLP class
We now start to build our function within a dedicated class.<br>

A brief reminder on the class object in Python:<br>
- Classes must have a function called _\_init_\_() which is automatically executed at class initiation
- Classes can have one or several methods
- Class object need to be instaciated before using its methods

#### A. The init function

This function aims at starting the session and setting up the desired service 

#### B. The close session method:
Simply kills the session so no ghost connection remains. 

#### C. The BDP method:
3 steps: <br>
1- Create request<br>
2- Send request <br>
3- Extract data<br>




In [3]:
class BLP():
    #-----------------------------------------------------------------------------------------------------    
    
    def __init__(self):               #ouvrir la session, si pas de bloomberg, ici va signer un pb
        """
            Improve this
            BLP object initialization
            Synchronus event handling
        """
        # Create Session object
        self.session = blpapi.Session()
        
        # Exit if can't start the Session
        if not self.session.start():
            print("Failed to start session.")
            return
        
        # Open & Get RefData Service or exit if impossible
        if not self.session.openService("//blp/refdata"):
            print("Failed to open //blp/refdata")
            return
        
        self.session.openService('//BLP/refdata')
        self.refDataSvc = self.session.getService('//BLP/refdata')

        print('Session open')
    
    #-----------------------------------------------------------------------------------------------------
    
    def bdh(self, strSecurity, strFields, startdate, enddate, per='DAILY', perAdj = 'CALENDAR', days = 'NON_TRADING_WEEKDAYS', fill = 'PREVIOUS_VALUE', curr = "EUR"):
        """
            Summary:
                HistoricalDataRequest ; 
        
                Gets historical data for a set of securities and fields

            Inputs:
                strSecurity: list of str : list of tickers
                strFields: list of str : list of fields, must be static fields (e.g. px_last instead of last_price)
                startdate: date
                enddate
                per: periodicitySelection; daily, monthly, quarterly, semiannually or annually
                perAdj: periodicityAdjustment: ACTUAL, CALENDAR, FISCAL
                curr: string, else default currency is used 
                Days: nonTradingDayFillOption : NON_TRADING_WEEKDAYS*, ALL_CALENDAR_DAYS or ACTIVE_DAYS_ONLY
                fill: nonTradingDayFillMethod :  PREVIOUS_VALUE, NIL_VALUE
                
                Options can be selected these are outlined in “Reference Services and Schemas Guide.”    
            
            Output:
                A list containing as many dataframes as requested fields
            # Partial response : 6
            # Response : 5
            
        """
           
        #-----------------------------------------------------------------------
        # Create request
        #-----------------------------------------------------------------------
        
        # Create request
        request = self.refDataSvc.createRequest('HistoricalDataRequest')
        
        # Put field and securities in list is single value is passed
        if type(strFields) == str:
            strFields = [strFields]
            
        if type(strSecurity) == str:
            strSecurity = [strSecurity]
    
        # Append list of securities
        for strF in strFields:
            request.append('fields', strF)
    
        for strS in strSecurity:
            request.append('securities', strS)
    
        # per='DAILY', perAdj = 'CALENDAR', days = 'NON_TRADING_WEEKDAYS', fill = 'PREVIOUS_VALUE', curr = None
        # Set other parameters
        request.set('startDate', startdate.strftime('%Y%m%d'))
        request.set('endDate', enddate.strftime('%Y%m%d'))
        request.set('periodicitySelection', per)
        request.set('nonTradingDayFillMethod',fill)
        request.set('nonTradingDayFillOption',days)
        request.set('currency',curr)
        #perAdj: periodicityAdjustment: ACTUAL, CALENDAR, FISCAL
        request.set('periodicityAdjustment',perAdj)
        
        #-----------------------------------------------------------------------
        # Send request
        #-----------------------------------------------------------------------
        
        requestID = self.session.sendRequest(request)
        print("Sending request")
        
        #-----------------------------------------------------------------------
        # Receive request
        #-----------------------------------------------------------------------
        
        dict_Security_Fields={}
        list_msg=[]
        #traitement du résultat
        while True:
            event = self.session.nextEvent()
            
            # Ignores anything that's not partial or final
            if (event.eventType() !=blpapi.event.Event.RESPONSE) & (event.eventType() !=blpapi.event.Event.PARTIAL_RESPONSE):
                #continue == on revient à "WHILE"
                continue
            
            # Extract the response message
            for msg in blpapi.event.MessageIterator(event):
            #msg = blpapi.event.MessageIterator(event).next()
                list_msg.append(msg)
                #print('---------------------------------------------------------')
                #print(msg)
                #print('---------------------------------------------------------')
                
            # Break loop if response is final
            if event.eventType() == blpapi.event.Event.RESPONSE:
                break        
        
        #-----------------------------------------------------------------------
        # Exploit data 
        #-----------------------------------------------------------------------
        
        
            # Add new key-value pair
            #dict_films['Marie Antoinette'] = 'Sofia Coppola'

        dictionary = dict()
        
        
        
        
        
        
        #print(list_msg)
        for msg in list_msg:
            #print(msg)           #print(msg.getElement(SECURITY_DATA))
            # getValue() pour print uniquement la valeur du champs
            
            #dictionary
            
            dictionary[msg.getElement(SECURITY_DATA).getElement(SECURITY).name()]=msg.getElement(SECURITY_DATA).getElement(SECURITY).getValue()
            print(msg.getElement(SECURITY_DATA).getElement(SECURITY).getValue())
            
            nom=[]
            for champs in msg.getElement(SECURITY_DATA).getElement(FIELD_DATA):
                for i in range(0,(champs.numElements())):
                    nom.append(champs.getElement(i).name())
            print(nom)
            
            """
            for champs in msg.getElement(SECURITY_DATA).getElement(FIELD_DATA):
                #print(champs)
                #print(champs.numElements())     #le nombre d'élément dans "champs"
                #print(champs.getElement(0))
                
                for i in range(0,(champs.numElements())):
                    #print(champs.getElement(i))
                    dictionary[champs.getElement(i).name()]=champs.getElement(i).getValue()
                    #print(champs.getElement(i).name())
                    #print(champs.getElement(i).getValue())
                # Loop on items
                for key, value in dictionary.items():
                    print('Element_Name:', key, ';   Value:', value)
                print('------------------------------------------------------')
            """
        return 0     
    
    #-----------------------------------------------------------------------------------------------------

    def closeSession(self):
        print("Session closed")
        self.session.stop()

In [4]:
class BLP_correction():
    #-----------------------------------------------------------------------------------------------------    
    
    def __init__(self):               #ouvrir la session, si pas de bloomberg, ici va signer un pb
        """
            Improve this
            BLP object initialization
            Synchronus event handling
        """
        # Create Session object
        self.session = blpapi.Session()
        
        # Exit if can't start the Session
        if not self.session.start():
            print("Failed to start session.")
            return
        
        # Open & Get RefData Service or exit if impossible
        if not self.session.openService("//blp/refdata"):
            print("Failed to open //blp/refdata")
            return
        
        self.session.openService('//BLP/refdata')
        self.refDataSvc = self.session.getService('//BLP/refdata')

        print('Session open')
    
    #-----------------------------------------------------------------------------------------------------
    
    def bdh(self, strSecurity, strFields, startdate, enddate, per='DAILY', perAdj = 'CALENDAR', days = 'NON_TRADING_WEEKDAYS', fill = 'PREVIOUS_VALUE', curr = "EUR"):
           
        #-----------------------------------------------------------------------
        # Create request
        #-----------------------------------------------------------------------
        
        # Create request
        request = self.refDataSvc.createRequest('HistoricalDataRequest')
        
        # Put field and securities in list is single value is passed
        if type(strFields) == str:
            strFields = [strFields]
            
        if type(strSecurity) == str:
            strSecurity = [strSecurity]
    
        # Append list of securities
        for strF in strFields:
            request.append('fields', strF)
    
        for strS in strSecurity:
            request.append('securities', strS)
    
        # per='DAILY', perAdj = 'CALENDAR', days = 'NON_TRADING_WEEKDAYS', fill = 'PREVIOUS_VALUE', curr = None
        # Set other parameters
        request.set('startDate', startdate.strftime('%Y%m%d'))
        request.set('endDate', enddate.strftime('%Y%m%d'))
        request.set('periodicitySelection', per)
        request.set('nonTradingDayFillMethod',fill)
        request.set('nonTradingDayFillOption',days)
        request.set('currency',curr)
        #perAdj: periodicityAdjustment: ACTUAL, CALENDAR, FISCAL
        request.set('periodicityAdjustment',perAdj)
        
        #-----------------------------------------------------------------------
        # Send request
        #-----------------------------------------------------------------------
        
        requestID = self.session.sendRequest(request)
        print("Sending request")
        
        #-----------------------------------------------------------------------
        # Receive request
        #-----------------------------------------------------------------------
        
        dict_Security_Fields={}
        list_msg=[]
        
        #creat as many empty dictionnaires as fields
        for field in strFields:
            globals()['dict_'+field]={}
        
        #traitement du résultat
        while True:
            event = self.session.nextEvent()
            
            # Ignores anything that's not partial or final
            if (event.eventType() !=blpapi.event.Event.RESPONSE) & (event.eventType() !=blpapi.event.Event.PARTIAL_RESPONSE):
                #continue == on revient à "WHILE"
                continue
            
            # Extract the response message
            for msg in blpapi.event.MessageIterator(event):
            #msg = blpapi.event.MessageIterator(event).next()
                list_msg.append(msg)
                #print('---------------------------------------------------------')
                #print(msg)
                #print('---------------------------------------------------------')
                
            # Break loop if response is final
            if event.eventType() == blpapi.event.Event.RESPONSE:
                break        
        
        #-----------------------------------------------------------------------
        # Exploit data 
        #-----------------------------------------------------------------------
        
        dict_output=dict()
        
        for msg in list_msg:
            ticker = str(msg.getElement(SECURITY_DATA).getElement(SECURITY).getValue())

            for field in strFields:
                globals()['dict_' + field][ticker] = {}
                
            for field_data in msg.getElement(SECURITY_DATA).getElement(FIELD_DATA):
                dat=field_data.getElement(DATE).getValue()
                for i in range(1,(field_data.numElements())):
                    field_name = str(field_data.getElement(i).name())
                    try:
                        globals()['dict_'+field_name][ticker][dat]=field_data.getElement(i).getValueAsFloat()
                    except:
                        globals()['dict_'+field_name][ticker][dat]=field_data.getElement(i).getValueAsString()
                    
        for field in strFields:
            dict_Security_Fields[field]=pd.DataFrame.from_dict(globals()['dict_'+field],orient = 'columns')
            
        return dict_Security_Fields
    
    #-----------------------------------------------------------------------------------------------------

    def closeSession(self):
        print("Session closed")
        self.session.stop()

# IV. Tests

In [5]:
blp = BLP_correction()
strFields = ["PX_LAST", "PX_VOLUME"]
tickers = ["GLE FP Equity", "TTE FP Equity"]
startDate = dt.datetime(2020,10,1) 
endDate = dt.datetime(2020,10,3)
dict_output = blp.bdh(strSecurity=tickers, strFields = strFields, startdate = startDate, enddate = endDate)

Session open
Sending request


In [7]:
dict_output['PX_LAST']

,GLE FP Equity,TTE FP Equity
2020-10-01,11.048,28.49
2020-10-02,11.038,28.21


In [ ]:
dict_output = dict()

strFields = ["AMT_OUTSTANDING", "PX_LAST"]

for msg in list_msg:
    security_data_array = msg.getElement(SECURITY_DATA)

    for i in range(security_data_array.numValues()):
        security_data = security_data_array.getValueAsElement(i)
        
        ticker = str(security_data.getElement(SECURITY).getValue())
        dict_output[ticker] = {}
        

            
            # 
            for field in strFields:
                if field_data.hasElement(field):
                    field_value = field_data.getElement(field).getValue()
                    dict_output[ticker][field] = field_value
                else:
                    print(f"{ticker}  '{field}' ")

            # ...


import pandas as pd
df = pd.DataFrame.from_dict(dict_output, orient='index')

# 打印 DataFrame
print(df)
